# Code-Hotspot-Analyse
### Step-by-Step Präsentation
Advanced Edition

### Rohdaten via Kommandozeile

#### Lines of Code

Einfache Variante mit `find`

In [ ]:
%%bash
cd spring-framework-petclinic/
find . -name *.java | xargs wc -l | head -n-1 > ../file_sizes.txt
head ../file_sizes.txt

Genauere Variante mit `cloc`

In [ ]:
%%bash
cloc spring-framework-petclinic/

Größeninformationen für jede Datei per `cloc`

In [ ]:
%%bash
cloc spring-framework-petclinic/ --by-file --quiet --csv --out lines.csv
head lines.csv

#### Änderungshäufigkeiten

Änderungen pro Datei mit `git log`

In [ ]:
%%bash
cd spring-framework-petclinic/
git log --name-only --no-merges --format="" -- *.java > ../changes.txt
cat ../changes.txt

### Datenmassage mit Data-Science-Werkzeugen

In der Theorie: Datei mit Zeilenanzahlen direkt einlesen

In [ ]:
!head -n 4 lines.txt

In [ ]:
import pandas as pd
pd.read_csv("lines.txt").head(3)

In der Praxis: Einlesen mit kleineren Anpassungen

In [ ]:
lines = pd.read_csv("lines.txt", index_col=1)[:-1][['code']]
lines.index = lines.index.str[2:]
lines.head()

Datei mit jeder geänderten Datei einlesen (+ Spaltennamen)

In [ ]:
!head -n 5 changes.txt

In [ ]:
change_per_file = pd.read_csv("changes.txt", names=['filepath'])
change_per_file.head()

Noch zu erledigen: Änderungen / Vorkommen der Dateien zählen

In [ ]:
changes = pd.DataFrame(change_per_file['filepath'].value_counts())
changes.columns = ["changes"]
changes.head()

Daten vereinen

In [ ]:
hotspots = changes.join(lines).dropna()
hotspots.head()

### Hinter den Kulissen der Visualisierung
Wir brauchen: Datenformat und Template

<small><pre>{
    'name': 'flare',
    'children': [{
            'name': 'src',
            'children': [{
                    'name': 'main',
                    'children': [{
                            'name': 'java',
                            'children': [{
                                    'name': 'org',
                                    'children': [{
                                            'name': 'springframework',
                                            'children': [{
                                                    'name': 'samples',
                                                    'children': [{
                                                            'name': 'petclinic',
                                                            'children': [{
                                                                    'name': 'repository',
                                                                    'children': [{
                                                                            'name': 'jdbc',
                                                                            'children': [{
                                                                                    'name': 'JdbcOwnerRepositoryImpl.java (158.0 [27])',
                                                                                    'size': 158.0,
                                                                                    'color': '#b40426'
                                                                                }, {
                                                                                    'name': 'JdbcVetRepositoryImpl.java (88.0 [20])',
                                                                                    'size': 88.0,
                                                                                    'color': '#f59d7e'
                                                                                }, {
                                                                                    'name': 'JdbcVisitRepositoryImpl.java (103.0 [19])',
                                                                                    'size': 103.0,
                                                                                    'color': '#f7aa8c'
                                                                                }, {
                                                                                    'name': 'JdbcPetRepositoryImpl.java (116.0 [17])',
                                                                                    'size': 116.0,
                                                                                    'color': '#f5c2aa'
                                                                                }, {
                                                                                    'name': 'JdbcPetRowMapper.java (40.0 [13])',
                                                                                    'size': 40.0,
                                                                                    'color': '#d8dce2'
                                                                                }, {
                                                                                    'name': 'JdbcPet.java (48.0 [8])',
                                                                                    'size': 48.0,
                                                                                    'color': '#9dbdff'
                                                                                }, {
                                                                                    'name': 'JdbcVisitRowMapper.java (40.0 [8])',
                                                                                    'size': 40.0,
                                                                                    'color': '#9dbdff'
                                                                                }, {
                                                                                    'name': 'JdbcPetVisitExtractor.java (53.0 [4])',
                                                                                    'size': 53.0,
                                                                                    'color': '#688aef'
                                                                                }, {
                                                                                    'name': 'OneToManyResultSetExtractor.java (159.0 [2])',
                                                                                    'size': 159.0,
                                                                                    'color': '#506bda'
                                                                                }, {
                                                                                    'name': 'package-info.java (6.0 [2])',
                                                                                    'size': 6.0,
                                                                                    'color': '#506bda'
                                                                                }
                                                                            ]
                                                                        }, {
                                                                            'name': 'jpa',
                                                                            'children': [{
                                                                                    'name': 'JpaOwnerRepositoryImpl.java (81.0 [17])',
                                                                                    'size': 81.0,
                                                                                    'color': '#f5c2aa'
                                                                                }, {
                                                                                    'name': 'JpaVisitRepositoryImpl.java (64.0 [12])',
                                                                                    'size': 64.0,
                                                                                    'color': '#cdd9ec'
                                                                                }, {
                                                                                    'name': 'JpaVetRepositoryImpl.java (48.0 [11])',
                                                                                    'size': 48.0,
                                                                                    'color': '#c3d5f4'
                                                                                }, {
                                                                                    'name': 'JpaPetRepositoryImpl.java (63.0 [10])',
                                                                                    'size': 63.0,
                                                                                    'color': '#b6cefa'
                                                                                }, {
                                                                                    'name': 'package-info.java (6.0 [2])',
                                                                                    'size': 6.0,
                                                                                    'color': '#506bda'
                                                                                }
                                                                            ]
                                                                        }
                                                                                ...
</pre></small>

Daten für Visualisierung aufbereiten

In [ ]:
from matplotlib import cm
from matplotlib.colors import rgb2hex

def create_plot_data(df, color_column_name, size_column_name, seperator):
    plot_data = pd.DataFrame(index=df.index)
    plot_data['value_for_color'] = df[color_column_name]
    plot_data['ratio_for_color'] = plot_data['value_for_color'] / plot_data['value_for_color'].max()
    plot_data['color'] = plot_data['ratio_for_color'].apply(lambda x : rgb2hex(cm.coolwarm(x)))
    plot_data['size'] = df[size_column_name]
    plot_data[['path', 'name']] = df.index.str.rsplit(seperator, n=1).to_list()
    plot_data['path_list'] = plot_data['path'].str.split(seperator)
    return plot_data

In D3-Datenformat (JSON) umwandeln

In [ ]:
import json

def create_flare_json(df):

    json_data = {'name': 'flare', 'children': []}

    for _, series in df.iterrows():
        hierarchical_data = series['path_list']

        children = json_data['children']
        for part in hierarchical_data:
            entry = next((child for child in children if child.get('name', '') == part), None)
            if not entry:
                entry = {'name': part, 'children': []}
                children.append(entry)
            children = entry['children']

        children.append({
            'name': f"{series['name']} ({series['size']} [{series['value_for_color']}])",
            'size': series['size'],
            'color': series['color']
        })

    return json_data

Alles anstoßen, HTML-Template mit Daten füttern und Datei erzeugen

In [ ]:
from IPython.core.display import HTML

def create_hotspot_file(hotspots, color_column_name, size_column_name, separator):
    json_data = create_flare_json(create_plot_data(hotspots, color_column_name, size_column_name,separator))
            
    with open("vis/template_hierarchical_d3_inline.html") as html_template:
        html = html_template.read().replace("###FLARE_JSON###", str(json_data))
        
        with open(f'hotspots.html', mode='w') as html_out:
            html_out.write(html)
    
    return HTML('<a href="hotspots.html">hotspots.html</a>')
            
create_hotspot_file(hotspots, "changes", "code", "/")

### Ende der Demo